# Actualizacion de Tablas SQL
## Utilizando las nuevas tablas desde el modulo de sistematizacion

Antes que todo, es necesario que la carpeta `db/CSV/` se encuentra con los archivos nuevos -i.e., copiar y reemplazar en la carpeta las tablas nuevas.

En particular, las nuevas tablas contienen procesada la columna `macro`. Esta columna asigna una etiqueta o categoria, permitiendo agrupar distintas respuestas. 

In [43]:
import pandas as pd
import numpy as np
import os 


from use_cases.pairs import create_pair_token
from use_cases.utils.textools import clean_alt_list, to_unicode, str_to_int, eliminate_nrs

from use_cases.contributions import create_table_contributions, to_sql as con_to_sql
from use_cases.emotions import create_table_emotions, to_sql as emo_to_sql 
from use_cases.personal_needs import create_table_personal_needs, to_sql as per_need_to_sql
from use_cases.country_needs import create_table_country_needs, to_sql as count_need_to_sql

### Emociones y Sentimientos

In [44]:
emotions_table = pd.read_csv('./db/CSV/emotions.csv', low_memory=False)

In [45]:
emotions_table.sample(2)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,is_online
21106,21106,enc_u_2801103,NaN,miedo,['miedo'],Miedo,-que los cambios son lentos. \n-seguridad de c...,"['cambios', 'lentos', 'seguridad', 'caleta']",True
74395,74395,NaN,ccf0cbafeca5c450763c5abd6ee1746a,euforica por el despertar un pueblo,"['euforica', ' despertar', ' pueblo']",Alegría,ver que el pueblo se manifesto para decir bast...,"['ver', 'pueblo', 'manifesto', 'decir', 'basta...",True


las listas se guardan como `strings` en el csv, es necesario volver a dejarlos como `listas`, asi tambien como pasar los id de los dialogos a `unicode`

In [46]:
emotions_table = eliminate_nrs(emotions_table)
emotions_table['name_tokens'] = emotions_table['name_tokens'].apply(lambda x: clean_alt_list(x))
emotions_table['exp_tokens'] = emotions_table['exp_tokens'].apply(lambda x: clean_alt_list(x))

In [47]:
emotions_table.sample(2)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,is_online
39847,39847,enc_u_4549300671199518306,,pena/angsustia,"[pena, angsustia]",Miedo,-ver como lo que se ha construido puede desapa...,"[ver, construido, puede, desaparecer, momento]",True
57534,57534,enc_u_4544819054211513783,,esperenza,[esperenza],Alegría,de que todo esto pase y alla una solucion,"[pase, alla, solucion]",True


Debemos recalcular los pares de palabras en caso de que algo haya cambiado

In [48]:
emotion_pair = create_pair_token(emotions_table, 'exp_tokens', 'emotion_id')

guardamos/reemplazamos la nueva tabla de pares y finalmente los nuevos `.sql`

In [49]:
emotion_pair.to_csv('./db/CSV/emotions_pairs.csv', index=False)
emo_to_sql(emotions_table, './db/SQL_INSERT/emotions.sql')

### Contribuciones

En el caso de contribuciones solo necesitamos guardar el nuevo `csv` como `sql`

In [50]:
contributions_table = pd.read_csv('./db/CSV/contributions.csv', low_memory=False)

In [51]:
contributions_table.sample(1)

,id,diag_id,ind_id,text,tokens,macro,is_online
65608,80920,NaN,4a8806a85101420eae877964a2ea73de,el activismo ciudadano y un trabajo constante ...,"['activismo', 'ciudadano', 'trabajo', 'constan...",Educación y autoeducación,True


In [52]:
contributions_table = eliminate_nrs(contributions_table)
contributions_table['tokens'] = contributions_table['tokens'].apply(lambda x: clean_alt_list(x))

In [53]:
contributions_table.sample(1)

,id,diag_id,ind_id,text,tokens,macro,is_online
20123,21530,enc_u_4563022110507654058,,"en nuestro ambito de trabajo, ensenando, trasm...","[ambito, trabajo, ensenando, trasmitiendo, con...",Educación y autoeducación,False


In [54]:
con_to_sql(contributions_table, './db/SQL_INSERT/contributions.sql')

### Necesidades Personales

In [55]:
personal_needs_table = pd.read_csv('./db/CSV/personal_needs.csv', low_memory=False)

In [56]:
personal_needs_table.sample(1)

,id,diag_id,ind_id,name,name_tokens,exp,exp_tokens,macro,priority,is_online
17702,17702,enc_u_4559528090111723216,NaN,adultos mayores,"['adultos', 'mayores']",miedo a quedarse solos,"['miedo', 'quedarse', 'solos']",trabajo,0,False


In [57]:
personal_needs_table = eliminate_nrs(personal_needs_table)

personal_needs_table['name_tokens'] = personal_needs_table['name_tokens'].apply(lambda x: clean_alt_list(x))
personal_needs_table['exp_tokens'] = personal_needs_table['exp_tokens'].apply(lambda x: clean_alt_list(x))


personal_needs_table['is_online'] = personal_needs_table['is_online'].apply(lambda x: bool(x))
personal_needs_table['priority'] = personal_needs_table['priority'].apply(lambda x: int(x))

In [58]:
personal_needs_table.sample(1)

,id,diag_id,ind_id,name,name_tokens,exp,exp_tokens,macro,priority,is_online
44500,44500,enc_u_4562983591194971878,,vivienda,[vivienda],"pocos cupos, han postulado muchas veces y no h...","[pocos, cupos, postulado, muchas, veces, conse...",vivienda,2,False


Debemos recalcular los pares de palabras en caso de que algo haya cambiado

In [59]:
personal_needs_pairs = create_pair_token(personal_needs_table, 'exp_tokens', 'personal_need_id')

guardamos/reemplazamos la nueva tabla de pares y finalmente los nuevos `.sql`

In [60]:
personal_needs_pairs.to_csv('./db/CSV/personal_needs_pairs.csv', index=False)
per_need_to_sql(personal_needs_table, './db/SQL_INSERT/personal_needs.sql')

### Necesidades Pais

In [61]:
country_needs_table = pd.read_csv('./db/CSV/country_needs.csv', low_memory=False)

In [62]:
country_needs_table.sample(1)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,role,role_tokens,actor,priority,is_online
136388,136398,NaN,4934725,calidad de vida,"['calidad', 'vida']",NR,NaN,NR,NaN,NaN,NaN,NaN,False


In [63]:
country_needs_table = eliminate_nrs(country_needs_table)
country_needs_table['name_tokens'] = country_needs_table['name_tokens'].apply(lambda x: clean_alt_list(x))
country_needs_table['exp_tokens'] = country_needs_table['exp_tokens'].apply(lambda x: clean_alt_list(x))
country_needs_table['role_tokens'] = country_needs_table['role_tokens'].apply(lambda x: clean_alt_list(x))

country_needs_table['is_online'] = country_needs_table['is_online'].apply(lambda x: bool(x))
country_needs_table['priority'] = country_needs_table['priority'].apply(lambda x: str_to_int(x))

In [65]:
country_needs_table.sample(1)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,role,role_tokens,actor,priority,is_online
138023,138033,,5d16b1bc7f35e885244c5c857799994c,justicia,[justicia],cambios politicos,expulsar a gente que hace dano a chile,"[expulsar, gente, hace, dano, chile]",jueces demaciado garantistas,"[jueces, demaciado, garantistas]",sistema penal,0,True


Debemos recalcular los pares de palabras en caso de que algo haya cambiado, en este caso tenemos dos tablas: exp y role

In [66]:
country_needs_exp_pairs_table = create_pair_token(country_needs_table, 'exp_tokens', 'country_need_id')
country_need_role_pair_table = create_pair_token(country_needs_table, 'role_tokens', 'country_need_id')

guardamos/reemplazamos la nueva tabla de pares y finalmente los nuevos .sql

In [67]:
country_needs_exp_pairs_table.to_csv('./db/CSV/country_needs_exp_pairs.csv', index=False)
country_need_role_pair_table.to_csv('./db/CSV/country_needs_role_pairs.csv', index=False)
count_need_to_sql(country_needs_table, './db/SQL_INSERT/country_needs.sql')